In [ ]:
from transformers import pipeline

# Loading pipeline with T5 model
summarizer = pipeline("summarization", model="t5-small")

C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('./output.csv')

In [3]:
data = df[2000:2100]

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(df, test_size=0.9, random_state=42)

In [4]:
import evaluate
rouge = evaluate.load("rouge")

Using the latest cached version of the module from C:\Users\shiva\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Sat Dec 14 15:57:24 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [5]:
data


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text
2000,2001,B001E5E29A,A1CPC3HEDIT8B5,"P. L. Carter ""historyteacher""",11,14,5,1178496000,Awesome,I usually make pancakes from scratch and have ...,summarize: i usually make pancakes from scratc...
2001,2002,B001E5E29A,A81HMEGGVESJP,"Mrs. G ""B. Real""",3,3,4,1207440000,Great tasting pancake mix,This is the best tasting pancake mix in the ma...,summarize: this is the best tasting pancake mi...
2002,2003,B001E5E29A,A13CUVB0LKBTB0,"Patricia A. Lukens ""Unifier""",5,6,5,1246838400,Stonewall Pancake Mix,My family absolutely loves this pancake and wa...,summarize: my family absolutely loves this pan...
2003,2004,B001E5E29A,A2Q8RE77HMDIK7,J. Remsen,2,2,5,1326585600,Delicious waffles!,We use the Farmhouse Pancake and Waffle Mix to...,summarize: we use the farmhouse pancake and wa...
2004,2005,B001E5E29A,A3VKPPHX72R4QI,"P. Mullen ""mullnc""",4,5,5,1215561600,Great taste and great price! A+++,I love Belgian waffles and this mix makes a de...,summarize: i love belgian waffles and this mix...
...,...,...,...,...,...,...,...,...,...,...,...
2095,2096,B007POA176,A22NBJNDK5JYWJ,raybabe,1,1,3,1343174400,They should say just add milk....,i got a bag of these and the veggie sticks. ha...,summarize: i got a bag of these and the veggie...
2096,2097,B007POA176,AFZW48UFK3G0H,C. Hancock,0,0,5,1347321600,Delicious,This was a new product to me. They were very ...,summarize: this was a new product to me. they ...
2097,2098,B007POA176,A830NL2LWO3TV,"S. Napolitano ""holistic health coach""",0,0,5,1338854400,Great new GMO-free snack!,"I was very pleased to stumble upon this brand,...",summarize: i was very pleased to stumble upon ...
2098,2099,B0001OINNQ,A2HJM83SBQXZJB,"Diane ""Madison Book lover""",0,0,5,1325980800,Perfect Gift,This product was a gift to a far distant relat...,summarize: this product was a gift to a far di...


In [7]:
def summarize(text):
    return summarizer(text, max_length=12, min_length=2, do_sample=False)[0]['summary_text']

In [8]:
data['summary'] = data['preprocessed_text'].apply(summarize)

C:\Users\shiva\AppData\Local\Temp\ipykernel_24848\1296683256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['summary'] = data['preprocessed_text'].apply(summarize)


In [9]:
results = rouge.compute(
    predictions=data["summary"].tolist(),
    references=data["Summary"].tolist()
)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.0983
rouge2: 0.0194
rougeL: 0.0932
rougeLsum: 0.0938


In [10]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [11]:

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
tokenizer = T5Tokenizer.from_pretrained("./Finetuned_T5/tokenizer")
model = T5ForConditionalGeneration.from_pretrained("./Finetuned_T5/model-2").to(device)

In [16]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=device)

In [19]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,summarize: i have bought several of the vitali...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,summarize: product arrived labeled as jumbo sa...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,summarize: this is a confection that has been ...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,summarize: if you are looking for the secret i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,summarize: great taffy at a great price. there...
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...,summarize: we switched from the advance simila...
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...","summarize: like the bad reviews say, the organ..."
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...,summarize: i wanted to solely breastfeed but w...
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...,summarize: i love the fact that i can get this...


In [20]:
new_data = df[2000:]  # Selecting data that my model don't know
new_data['summary'] = new_data['preprocessed_text'].apply(summarize)

Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors
C:\Users\shiva\AppData\Local\Temp\ipykernel_24848\4149046823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['summary'] = new_data['preprocessed_text'].apply(summarize)


In [21]:
results = rouge.compute(
    predictions=new_data["summary"].tolist(),
    references=new_data["Summary"].tolist()
)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1450
rouge2: 0.0457
rougeL: 0.1430
rougeLsum: 0.1428


On just training on 800 examples our model got better. So we can proceeded to train on huge examples